In [1]:
from dataclasses import dataclass
from typing import Optional, List

@dataclass
class NewsArticle:
    category: str
    title: str
    text: str

@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

def read_news_file(filepath: str) -> List[NewsArticle]:
    articles = []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for _, line in enumerate(f, 1):
            line = line.rstrip('\n')
            category, title, text = line.split('\t')
            article = NewsArticle(
                category=category.strip(),
                title=title.strip(),
                text=text.strip()
            )
            
            articles.append(article)
    
    return articles
filepath = 'data/news.txt'
articles = read_news_file(filepath)
if articles:
    print("\nПример первой статьи:")
    print(f"Категория: {articles[0].category}")
    print(f"Заголовок: {articles[0].title}")
    print(f"Текст: {articles[0].text}")


Пример первой статьи:
Категория: style
Заголовок: Rolex наградит победителей регаты
Текст: Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных 

In [ ]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, is_capitalized
from yargy.interpretation import fact

Person = fact(
    "Person",
    ["first", "middle", "last"]
)

# Паттерн 1: Имя + Фамилия
NAME_SURNAME = rule(
    gram("Name").interpretation(Person.first.inflected()),
    gram("Surn").interpretation(Person.last.inflected())
).interpretation(Person)

# Паттерн 2: Имя + Отчество + Фамилия
NAME_PATRONYMIC_SURNAME = rule(
    gram("Name").interpretation(Person.first.inflected()),
    gram("Patr").interpretation(Person.middle.inflected()),
    gram("Surn").interpretation(Person.last.inflected())
).interpretation(Person)

# Паттерн 3: Только фамилия
SURNAME_ONLY = rule(
    gram("Surn").interpretation(Person.last.inflected())
).interpretation(Person)

# Паттерн 4: Иностранные имена
FOREIGN_NAME = rule(
    is_capitalized().interpretation(Person.first),
    is_capitalized().interpretation(Person.last)
).interpretation(Person)

PERSON = or_(
    NAME_PATRONYMIC_SURNAME,
    NAME_SURNAME,
    FOREIGN_NAME,
    SURNAME_ONLY
).interpretation(Person)

In [ ]:
parser = Parser(PERSON)

test_texts = [
    "Владимир Путин выступил с речью",
    "Сергей Сергеевич Иванов прибыл в Москву",
    "По словам Медведева, ситуация улучшается",
    "Дональд Трамп встретился с Джо Байденом",
    "Президент России Владимир Владимирович Путин",
    "В. В. Путин принял участие в саммите",
    "А. Навальный выступил с заявлением",
    "Министр иностранных дел Лавров",
    "Американский президент Обама подписал указ",
    "Канцлер Меркель встретилась с премьером",
    "Иванов Сергей получил награду",
]

print("Тестирование расширенных паттернов для поиска имен:\n")
print("="*80)
for i, text in enumerate(test_texts, 1):
    print(f"\n{i}. Текст: {text}")
    matches = list(parser.findall(text))
    if matches:
        for match in matches:
            fact = match.fact
            print(f"   Найдено: {match.fact}")
            print(f"   Токены: {[_.value for _ in match.tokens]}")
    else:
        print("Ничего не найдено")
print("\n" + "="*80)

Тестирование расширенных паттернов для поиска имен:


1. Текст: Владимир Путин выступил с речью
   Найдено: Person(first='владимир', middle=None, last='путин')
   Токены: ['Владимир', 'Путин']

2. Текст: Сергей Сергеевич Иванов прибыл в Москву
   Найдено: Person(first='сергей', middle='сергеевич', last='иванов')
   Токены: ['Сергей', 'Сергеевич', 'Иванов']

3. Текст: По словам Медведева, ситуация улучшается
   Найдено: Person(first=None, middle=None, last='по')
   Токены: ['По']
   Найдено: Person(first=None, middle=None, last='медведев')
   Токены: ['Медведева']

4. Текст: Дональд Трамп встретился с Джо Байденом
   Найдено: Person(first='дональд', middle=None, last='трамп')
   Токены: ['Дональд', 'Трамп']
   Найдено: Person(first='Джо', middle=None, last='Байденом')
   Токены: ['Джо', 'Байденом']

5. Текст: Президент России Владимир Владимирович Путин
   Найдено: Person(first='Президент', middle=None, last='России')
   Токены: ['Президент', 'России']
   Найдено: Person(first='владимир

c:\Users\tizza\Projects\nlp_homework\task1\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, dictionary, normalized, gte, lte, eq, type as type_
from yargy.interpretation import fact
from yargy.pipelines import morph_pipeline

BirthDate = fact(
    "BirthDate",
    ["day", "month", "year"]
)

DAY = and_(
    type_("INT"),
    gte(1),
    lte(31)
).interpretation(BirthDate.day)

YEAR = and_(
    type_("INT"),
    gte(1000),
    lte(2100)
).interpretation(BirthDate.year)

MONTH = morph_pipeline([
    "январь",
    "февраль", 
    "март",
    "апрель",
    "май",
    "июнь",
    "июль",
    "август",
    "сентябрь",
    "октябрь",
    "ноябрь",
    "декабрь"
]).interpretation(BirthDate.month.inflected())

# Маркеры рождения
BIRTH_MARKER = or_(
    morph_pipeline(["родиться"]),
    morph_pipeline(["родить"]),     
    rule(normalized("рождение")),     
    rule(normalized("род.")),       
    rule(normalized("год"), normalized("рождение"))
)

# Паттерн 1: родился/родилась + день + месяц + год
BIRTH_FULL_DATE = rule(
    BIRTH_MARKER,
    DAY,
    MONTH,
    YEAR,
    normalized("год").optional()
).interpretation(BirthDate)

# Паттерн 2: родился + месяц + год
BIRTH_MONTH_YEAR = rule(
    BIRTH_MARKER,
    normalized("в").optional(),
    MONTH,
    YEAR,
    normalized("год").optional()
).interpretation(BirthDate)

# Паттерн 3: родился + только год
BIRTH_YEAR_ONLY = rule(
    BIRTH_MARKER,
    normalized("в").optional(),
    YEAR,
    or_(
        normalized("год"),
        normalized("г.")
    ).optional()
).interpretation(BirthDate)

# Паттерн 4: год рождения - XXXX
BIRTH_YEAR_EXPLICIT = rule(
    normalized("год"),
    normalized("рождение"),
    or_(
        eq("-"),
        eq(":"),
        eq("—")
    ).optional(),
    YEAR
).interpretation(BirthDate)

# Паттерн 5: в скобках (род. 1980)
BIRTH_PARENTHESIS = rule(
    eq("("),
    or_(
        morph_pipeline(["род."]),
        morph_pipeline(["р."])
    ),
    YEAR,
    eq(")").optional()
).interpretation(BirthDate)

# Паттерн 6: числовая дата DD.MM.YYYY
NUMERIC_DATE = rule(
    DAY,
    eq("."),
    and_(type_("INT"), gte(1), lte(12)).interpretation(BirthDate.month),
    eq("."),
    YEAR
).interpretation(BirthDate)

BIRTHDATE = or_(
    BIRTH_FULL_DATE,
    BIRTH_MONTH_YEAR,
    NUMERIC_DATE,
    BIRTH_PARENTHESIS,
    BIRTH_YEAR_EXPLICIT,
    BIRTH_YEAR_ONLY
).interpretation(BirthDate)

In [ ]:
parser = Parser(BIRTHDATE)

test_texts = [
    "Пётр Ильич Чайковский родился 7 мая 1840 года",
    "Толстой родился в августе 1828 года",
    "Пушкин родился в 1799 году",
    "Иванов (род. 1985) работает директором",
    "Год рождения - 1990",
    "Родившийся 15 марта 1985 года Петров",
    "Дата рождения: 01.01.2000",
    "Она родилась в мае 1975",
    "Композитор родился в 1756",
]

print("Тестирование паттернов для поиска дат рождения:\n")
print("="*80)
for i, text in enumerate(test_texts, 1):
    print(f"\n{i}. Текст: {text}")
    matches = list(parser.findall(text))
    if matches:
        for match in matches:
            print(f"   Найдено: {match.fact}")
            print(f"   Токены: {[_.value for _ in match.tokens]}")
    else:
        print("Ничего не найдено")
print("\n" + "="*80)

Тестирование паттернов для поиска дат рождения:


1. Текст: Пётр Ильич Чайковский родился 7 мая 1840 года
   Найдено: BirthDate(day='7', month='май', year='1840')
   Токены: ['родился', '7', 'мая', '1840', 'года']

2. Текст: Толстой родился в августе 1828 года
   Найдено: BirthDate(day=None, month='август', year='1828')
   Токены: ['родился', 'в', 'августе', '1828', 'года']

3. Текст: Пушкин родился в 1799 году
   Найдено: BirthDate(day=None, month=None, year='1799')
   Токены: ['родился', 'в', '1799', 'году']

4. Текст: Иванов (род. 1985) работает директором
   Найдено: BirthDate(day=None, month=None, year='1985')
   Токены: ['(', 'род', '.', '1985', ')']

5. Текст: Год рождения - 1990
   Найдено: BirthDate(day=None, month=None, year='1990')
   Токены: ['Год', 'рождения', '-', '1990']

6. Текст: Родившийся 15 марта 1985 года Петров
   Найдено: BirthDate(day='15', month='март', year='1985')
   Токены: ['Родившийся', '15', 'марта', '1985', 'года']

7. Текст: Дата рождения: 01.01.2000
  

In [ ]:
from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, normalized, is_capitalized, eq
from yargy.interpretation import fact
from yargy.pipelines import morph_pipeline

BirthPlace = fact(
    "BirthPlace",
    ["location"]
)

BIRTHPLACE_MARKER = or_(
    morph_pipeline(["родиться"])
    morph_pipeline(["уроженец"]),
    morph_pipeline(["уроженка"]),
    rule(normalized("место"), normalized("рождение")),
    rule(normalized("родом"), normalized("из"))
)

GEO_TYPE = morph_pipeline([
    "город",
    "деревня", 
    "село",
    "поселок",
    "столица",
    "область",
    "край",
    "регион",
    "республика",
    "провинция",
    "штат",
    "страна",
    "государство",
    "империя",
    "королевство"
])


LOCATION_NAME = and_(
    is_capitalized(),
    or_(
        gram("NOUN"),
        gram("ADJF")
    )
).interpretation(BirthPlace.location)

LOCATION_MULTI = rule(
    LOCATION_NAME,
    LOCATION_NAME.optional()
)

# Паттерн 1: родился в Городе
BIRTHPLACE_SIMPLE = rule(
    BIRTHPLACE_MARKER,
    normalized("в").optional(),
    GEO_TYPE.optional(),
    LOCATION_MULTI
).interpretation(BirthPlace)

# Паттерн 2: уроженец Города
BIRTHPLACE_NATIVE = rule(
    or_(
        morph_pipeline(["уроженец"]),
        morph_pipeline(["уроженка"])
    ),
    GEO_TYPE.optional(),
    LOCATION_MULTI
).interpretation(BirthPlace)

# Паттерн 3: место рождения - Город
BIRTHPLACE_EXPLICIT = rule(
    normalized("место"),
    normalized("рождение"),
    or_(
        eq("-"),
        eq(":"),
        eq("—")
    ).optional(),
    GEO_TYPE.optional(),
    LOCATION_MULTI
).interpretation(BirthPlace)

# Паттерн 4: родом из Города
BIRTHPLACE_ORIGIN = rule(
    normalized("родом"),
    normalized("из"),
    GEO_TYPE.optional(),
    LOCATION_MULTI
).interpretation(BirthPlace)

# Паттерн 5: в скобках с городом
BIRTHPLACE_PARENTHESIS = rule(
    eq("("),
    or_(
        rule(
            or_(normalized("род."), normalized("р.")),
            and_(gram("INT"), gte(1000), lte(2100)),
            eq(","),
            LOCATION_MULTI
        ),
        rule(
            LOCATION_MULTI,
            eq(","),
            and_(gram("INT"), gte(1000), lte(2100))
        )
    ),
    eq(")").optional()
).interpretation(BirthPlace)

BIRTHPLACE = or_(
    BIRTHPLACE_EXPLICIT,
    BIRTHPLACE_NATIVE,
    BIRTHPLACE_ORIGIN,
    BIRTHPLACE_SIMPLE
).interpretation(BirthPlace)

Тестирование паттернов для поиска мест рождения:


1. Текст: Пётр Ильич Чайковский родился в городе Воткинск
   Найдено: BirthPlace(location='Воткинск')
   Токены: ['родился', 'в', 'городе', 'Воткинск']

2. Текст: Толстой родился в Ясной Поляне
   Найдено: BirthPlace(location='Поляне')
   Токены: ['родился', 'в', 'Ясной', 'Поляне']

3. Текст: Пушкин родился в Москве
   Найдено: BirthPlace(location='Москве')
   Токены: ['родился', 'в', 'Москве']

4. Текст: Уроженец Санкт-Петербурга Иванов
   Найдено: BirthPlace(location='Санкт')
   Токены: ['Уроженец', 'Санкт']

5. Текст: Место рождения - Париж
   Найдено: BirthPlace(location='Париж')
   Токены: ['Место', 'рождения', '-', 'Париж']

6. Текст: Он родом из Берлина
   Найдено: BirthPlace(location='Берлина')
   Токены: ['родом', 'из', 'Берлина']

7. Текст: Родился в столице Франции
   Найдено: BirthPlace(location='Франции')
   Токены: ['Родился', 'в', 'столице', 'Франции']

8. Текст: Композитор, уроженка Вены, выступила с концертом
   Найден

In [ ]:
parser = Parser(BIRTHPLACE)

test_texts = [
    "Пётр Ильич Чайковский родился в городе Воткинск",
    "Толстой родился в Ясной Поляне",
    "Пушкин родился в Москве",
    "Уроженец Санкт-Петербурга Иванов",
    "Место рождения - Париж",
    "Он родом из Берлина",
    "Родился в столице Франции",
    "Композитор, уроженка Вены, выступила с концертом",
    "Петров (уроженец Киева) получил награду",
    "Она родилась в деревне Константиново",
]

print("Тестирование паттернов для поиска мест рождения:\n")
print("="*80)
for i, text in enumerate(test_texts, 1):
    print(f"\n{i}. Текст: {text}")
    matches = list(parser.findall(text))
    if matches:
        for match in matches:
            print(f"   Найдено: {match.fact}")
            print(f"   Токены: {[_.value for _ in match.tokens]}")
    else:
        print("   ❌ Ничего не найдено")
print("\n" + "="*80)

In [7]:
from yargy.interpretation import fact
from typing import Optional, List
from collections import defaultdict

Person = fact("Person", ["first", "middle", "last"])
BirthDate = fact("BirthDate", ["day", "month", "year"])
BirthPlace = fact("BirthPlace", ["location"])

person_parser = Parser(PERSON)
birthdate_parser = Parser(BIRTHDATE)
birthplace_parser = Parser(BIRTHPLACE)


def format_person_name(fact):
    """Форматирует имя из fact в строку"""
    parts = []
    if fact.first:
        parts.append(fact.first)
    if fact.middle:
        parts.append(fact.middle)
    if fact.last:
        parts.append(fact.last)
    return " ".join(parts)

def format_birth_date(fact):
    """Форматирует дату рождения из fact в строку"""
    parts = []
    if fact.day:
        parts.append(str(fact.day))
    if fact.month:
        parts.append(str(fact.month))
    if fact.year:
        parts.append(str(fact.year))
    return " ".join(parts) if parts else None

def format_birth_place(fact):
    """Форматирует место рождения из fact в строку"""
    return fact.location if fact.location else None

def read_news_file(filepath: str) -> List[NewsArticle]:
    """Читает файл с новостями"""
    articles = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.rstrip('\n')
            parts = line.split('\t')
            if len(parts) == 3:
                article = NewsArticle(
                    category=parts[0].strip(),
                    title=parts[1].strip(),
                    text=parts[2].strip()
                )
                articles.append(article)
    return articles

def extract_entries_from_text(text: str) -> List[Entry]:
    """Извлекает Entry из текста"""
    entries = []
    
    person_matches = list(person_parser.findall(text))
    birthdate_matches = list(birthdate_parser.findall(text))
    birthplace_matches = list(birthplace_parser.findall(text))

    for person_match in person_matches:
        name = format_person_name(person_match.fact)
        
        birth_date = None
        if birthdate_matches:
            birth_date = format_birth_date(birthdate_matches[0].fact)
        
        birth_place = None
        if birthplace_matches:
            birth_place = format_birth_place(birthplace_matches[0].fact)
        
        entry = Entry(
            name=name,
            birth_date=birth_date,
            birth_place=birth_place
        )
        entries.append(entry)
    
    return entries

Загрузка данных...
Загружено 10000 статей

Извлечение данных...
Обработано 1000/10000 статей...
Обработано 2000/10000 статей...
Обработано 3000/10000 статей...
Обработано 4000/10000 статей...
Обработано 5000/10000 статей...
Обработано 6000/10000 статей...
Обработано 7000/10000 статей...
Обработано 8000/10000 статей...
Обработано 9000/10000 статей...
Обработано 10000/10000 статей...

СТАТИСТИКА:
Всего найдено записей: 105531
С датой рождения: 982
С местом рождения: 1063
С датой И местом: 29

Примеры найденных записей (первые 20):

1. Имя: Giraglia Rolex
   Дата рождения: None
   Место рождения: None

2. Имя: Yacht Club
   Дата рождения: None
   Место рождения: None

3. Имя: Rolex Yacht
   Дата рождения: None
   Место рождения: None

4. Имя: Mercury Соревнования
   Дата рождения: None
   Место рождения: None

5. Имя: по
   Дата рождения: None
   Место рождения: None

6. Имя: по
   Дата рождения: None
   Место рождения: None

7. Имя: по
   Дата рождения: None
   Место рождения: None

8. И

In [ ]:
print("Загрузка данных...")
filepath = 'data/news.txt'
articles = read_news_file(filepath)
print(f"Загружено {len(articles)} статей\n")

print("Извлечение данных...")
all_entries = []
stats = defaultdict(int)

for i, article in enumerate(articles):
    if (i + 1) % 1000 == 0:
        print(f"Обработано {i + 1}/{len(articles)} статей...")
    
    # Извлекаем из заголовка и текста
    text = article.title + " " + article.text
    entries = extract_entries_from_text(text)
    
    for entry in entries:
        all_entries.append(entry)
        stats['total_entries'] += 1
        if entry.birth_date:
            stats['with_date'] += 1
        if entry.birth_place:
            stats['with_place'] += 1
        if entry.birth_date and entry.birth_place:
            stats['with_both'] += 1

print(f"\n{'='*80}")
print("СТАТИСТИКА:")
print(f"{'='*80}")
print(f"Всего найдено записей: {stats['total_entries']}")
print(f"С датой рождения: {stats['with_date']}")
print(f"С местом рождения: {stats['with_place']}")
print(f"С датой И местом: {stats['with_both']}")
print(f"{'='*80}\n")

print("Примеры найденных записей (первые 20):")
print(f"{'='*80}")
for i, entry in enumerate(all_entries[:20], 1):
    print(f"\n{i}. Имя: {entry.name}")
    print(f"   Дата рождения: {entry.birth_date}")
    print(f"   Место рождения: {entry.birth_place}")
print(f"\n{'='*80}")